# Hybrid quantum-classical auto encoder 

In [ ]:
"""
Sophie Choe sophchoe@pdx.edu
Portland State University
9/24/2021
"""

In [ ]:
"""
This is an implementation of the hybrid quantum-classical auto-encoder model outlined in "Continuous-Variable 
quantum neural networks" Physical Review Research 1, 033063, 2019. The encoder consists of six classical neural 
network layers and the decoder consists of three quantum neural network layers. Input vectors of length 3 are 
encoded in vectors of length 2, they are used as parameters of the Displacement gate paremeters, followed by the 
quantum decoder producing output vectors of length 3. 

The classical layers are implemented with Keras. The quantum layers are implemented with Pennylane. With the 
Pennylane qml.qnn.KerasLayer plug-in, the whole network is converted to a Keras model, and Keras loss function 
and optimizer are used for training.

The loss function in the paper requires state vectors and probability, however Pennylane measurement module does 
not support state vector retrieval (as of 9/24/2021). Hence mean squared error from Keras is used instead on the 
output vectors of probability measurement. To get vectors of length 3, the cutoff dimension parameter is set to 3. 

It reaches around 70% training accuracy and around 75% evaluation accuracy
"""

Dependencies: anaconda-navigator==1.10.0 keras-nightly==2.5.0.dev2021032900 PennyLane==0.17.0 StrawberryFields==0.18.0 tensorflow @ file:///Users/uwe/miniconda3/conda-bld/tensorflow-split_1618075966251/work/tensorflow_pkg/tensorflow-2.4.0-cp38-cp38-macosx_10_9_x86_64.whl

In [2]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

import pennylane as qml
from pennylane import numpy as np

In [3]:
# ===================================================================================
#                       Create Data
# ===================================================================================

num_train = 100
num_test = 20
len_vector = 3

np.random.seed(1)
x_train = np.random.rand(num_train, len_vector)
x_test = np.random.rand(num_test, len_vector)

In [4]:
# ===================================================================================
#                       Classical Layers using Keras Sequential
# ===================================================================================

# Define first layer, hidden layers, and output layer with the output of two neurons

keras.backend.set_floatx('float32')
model = tf.keras.Sequential(
                            [layers.Dense(5, input_shape=(3,), activation ="elu"),
                             layers.Dense(5, activation ="elu"),
                             layers.Dense(5, activation ="elu"),
                             layers.Dense(5, activation ="elu"),
                             layers.Dense(5, activation ="elu"),
                             layers.Dense(5, activation ="elu"),
                             layers.Dense(2, activation ="elu")]
                            )

In [5]:
# ===================================================================================
#                                Quantum Iterative Layer
# ===================================================================================

# Initialize weights to pass through the q_layer function. Tensor of size: num_layers x num_parameters  
# Params for the Rotation gate, Squeezing gate, Displacement gate, and Kerr gate

def init_weights(layers, modes, active_sd=0.0001, passive_sd=0.1):
    
    r1_weights = tf.random.normal(shape=[layers, modes], stddev=passive_sd)
    s_weights = tf.random.normal(shape=[layers, modes], stddev=active_sd)
    r2_weights = tf.random.normal(shape=[layers, modes], stddev=passive_sd)
    dr_weights = tf.random.normal(shape=[layers, modes], stddev=active_sd)
    k_weights = tf.random.normal(shape=[layers, modes], stddev=active_sd)

    weights = tf.concat([r1_weights, s_weights, r2_weights, dr_weights, k_weights], axis=1)
    weights = tf.Variable(weights)

    return weights

# Construct a quantum layer with initialized weights as variables

def q_layer(v):
    qml.Rotation(v[0], wires=0)
    qml.Squeezing(v[1], 0.0, wires=0)
    qml.Rotation(v[2], wires=0)
    qml.Displacement(v[3], 0.0, wires=0)
    qml.Kerr(v[4], wires=0)

In [6]:
# ===================================================================================
#                                Quantum Circuit
# ===================================================================================

# Use the output of the classical layers to initialize the quantum layers with the Displacement gate
# Iterate the quantum layers

num_modes = 1
num_basis = 3

dev = qml.device("strawberryfields.fock", wires=num_modes, cutoff_dim=num_basis) # select a devide 

@qml.qnode(dev, interface = "tf")
def quantum_nn(inputs, var):
    
    # Inputs: the output of classical layers as displacement gate parameters
    qml.Displacement(inputs[0], inputs[1], wires=0)    

    # quantum layers
    for v in var:
        q_layer(v)

    return qml.probs(wires=0)

In [7]:
# ===================================================================================
#                             Hybrid Model
# ===================================================================================

"""
Add the quantum layer to the classical to create a hybrid model
    1. initialize weights for quantum layers
    2. create a dictionary of weight shape to pass as one of the variables to covert to keras layer
    3. convert the quantum layer to a Keras layer
    4. add to the classical sequential model
"""

num_layers = 4
num_modes = 1

weigths = init_weights(num_layers, num_modes)
shape_tup = weigths.shape
weight_shapes = {'var': shape_tup}
qlayer = qml.qnn.KerasLayer(quantum_nn, weight_shapes, output_dim=4)
model.add(qlayer)

In [8]:
# ===================================================================================
#                           Loss Function and Optimizer
# ===================================================================================

loss = tf.keras.losses.MeanSquaredError()
opt = keras.optimizers.Adam(learning_rate=0.01)
#opt = tf.keras.optimizers.SGD(learning_rate=0.02)

model.compile(opt, loss=loss, metrics=["accuracy"])

In [ ]:
# ===================================================================================
#                                    Training
# ===================================================================================

model.fit(x_train, 
          x_train,
          epochs=200,
          batch_size=24,
          shuffle=True,
          validation_data=(x_test, x_test))

Epoch 1/200


In [58]:
# ===================================================================================
#                                    Evalutation
# ===================================================================================

results = model.evaluate(x_test, x_test, batch_size=128)
print("test loss, test acc:", results)

1/1 [==============================] - 0s 146ms/step - loss: 0.0941 - accuracy: 0.7500
test loss, test acc: [0.09412109851837158, 0.75]
